In [48]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
%aimport exv2.experiment_results 

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import glasbey



pd.set_option('display.max_columns', None)
sns.set_theme(rc={'figure.figsize':(12, 6)})
sns.set_context("paper")
sns.set_style("whitegrid")

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

palette = glasbey.create_block_palette(
    [4, 3, 3, 2, 2],
    colorblind_safe=True,
    cvd_severity=90
)
sns.set_palette(palette)

dep_focus = ["baseline_vanilla_full","monolith_feature_monolith_full","serverless_feature_serverless_full"]
ms_focus = ["baseline_vanilla_full","obs_feature_object-storage_full","norec_feature_norecommendations_full"]
lable_names = {
    "baseline_vanilla_full": "Microservice Baseline",
    'jvm_jvm-impoove_full': "Runtime Improvement",
    'monolith_feature_monolith_full': "Monolith Variant",
    'norec_feature_norecommendations_full' : "Service Reduction",
    'obs_feature_object-storage_full' : "Infrastructure Service Replacement",
    'serverless_feature_serverless_full' : "Serverless Variant",
}

In [50]:
exr = exv2.experiment_results.ExperimentResults("data/2024-07-19*", load_stats_history=True) # type: ignore
RUN_VARS = exv2.experiment_results.ExperimentResults.RUN_VARS # type: ignore

assert("loadgenerator" not in (list(exr.pods['name'].unique())))
assert("unkown" not in (list(exr.pods['instance'].unique())))


/Users/b/projects/teastore_experiments/exv2/experiment_results.py:171: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pods = pd.concat(


In [51]:
# AWS cost-model

serverless_price = 0.0000166667  # based on aws lambda price per GB-s  (frankfurt)
memory_second_price = 0.00511 / 1024 /60 # $/MBs based on AWS nfragate memory price per hour (frankfurt) 
vCPU_second_price = 0.04656 / 60 # $/vCPU based on AWS nfragate memory price per hour (frankfurt) 

pod_configuration = {
    "teastore-recommender": {"cpu": 2600, "memory": 1332},
    "teastore-webui": {"cpu": 1300, "memory": 1950},
    "teastore-image": {"cpu": 1300, "memory": 1950},
    "teastore-auth": {"cpu": 585, "memory": 1332},
    'teastore-registry':{"cpu": 1000, "memory": 1024}, # not set by default ....
    'teastore-persistence':{"cpu": 1000, "memory": 1024}, # not set by default ....
    'teastore-db':{"cpu": 1000, "memory": 1024}, # not set by default ....
    "teastore-all": {"cpu":1950, "memory":2663},
    "auth": {"cpu": 500, "memory": 500},
}

import numpy as np
# we calculate the cost for each pod based on the configuration and the time it was running
def calc_request_based_billing(row):
    if row["type"] == "pod":
        conf = pod_configuration[row["pod_name"]] 
        return conf["memory"] * memory_second_price + np.ceil(conf["cpu"]/1000) * vCPU_second_price
    elif row["type"] == "function":
        return 500*serverless_price
    

def calc_usage_based_billing(row):
    if row["type"] == "pod":
        return row["memory_usage"] * memory_second_price + np.ceil(row["cpu_usage"]) * vCPU_second_price
    elif row["type"] == "function":
        return row["memory_usage"]*serverless_price


In [52]:
pods = exr.pods
pods["pod_name"] = pods["name"].apply(lambda x: "-".join(x.split("-")[0:2]))

pods["type"] = pods["pod_name"].apply(lambda x: "pod" if x.startswith("teastore") else "function" if x.startswith("auth") else "infra")
# ignore infra pods for now
pods = pods[pods["type"].isin(["pod","function"])]
pods_usage = pods.groupby(exr.RUN_VARS+["run_time","name","pod_name","type"])[["memory_usage","cpu_usage"]].sum().reset_index()

pods_usage["requested_cost"] = pods_usage.apply(calc_request_based_billing, axis=1)
pods_usage["used_cost"] = pods_usage.apply(calc_usage_based_billing, axis=1)



In [53]:
pods_mean_cost = pods_usage.groupby(exr.RUN_VARS)[["requested_cost","used_cost"]].sum().reset_index().groupby(["exp_branch","exp_workload"])[["requested_cost","used_cost"]].mean().reset_index()
pods_mean_cost

,exp_branch,exp_workload,requested_cost,used_cost
0,baseline_vanilla_full,exp_scale_fixed,0.048904,0.024746
1,baseline_vanilla_full,exp_scale_rampup,0.820686,0.371251
2,baseline_vanilla_full,exp_scale_shaped,0.578921,0.278816
3,jvm_jvm-impoove_full,exp_scale_fixed,0.037350,0.013179
4,jvm_jvm-impoove_full,exp_scale_pausing,0.578921,0.269267
5,jvm_jvm-impoove_full,exp_scale_rampup,0.822756,0.404496
6,jvm_jvm-impoove_full,exp_scale_shaped,0.578921,0.267598
7,monolith_feature_monolith_full,exp_scale_fixed,0.010539,0.004126
8,monolith_feature_monolith_full,exp_scale_pausing,0.163348,0.078036
9,monolith_feature_monolith_full,exp_scale_rampup,0.259978,0.107194


In [70]:
requests = exr.stats.groupby(["exp_branch","exp_workload"])[["Request Count","Failure Count"]].sum().reset_index() # total request count
requests["rq"] = requests["Request Count"] - requests["Failure Count"]

pods_mean_cost_per_request = pods_mean_cost.merge(requests[["exp_branch","exp_workload","rq"]], on=["exp_branch","exp_workload"])
pods_mean_cost_per_request["requested_cost_per_r"] = (pods_mean_cost_per_request["requested_cost"] / pods_mean_cost_per_request["rq"]) * 100 * 1000 # convert to mili cents
pods_mean_cost_per_request["used_cost_per_r"] = (pods_mean_cost_per_request["used_cost"] / pods_mean_cost_per_request["rq"]) * 100 * 1000 # convert to mili cents

pods_mean_cost_per_request

,exp_branch,exp_workload,requested_cost,used_cost,rq,requested_cost_per_r,used_cost_per_r
0,baseline_vanilla_full,exp_scale_fixed,0.048904,0.024746,367,13.325372,6.742670
1,baseline_vanilla_full,exp_scale_rampup,0.820686,0.371251,56169,1.461102,0.660954
2,baseline_vanilla_full,exp_scale_shaped,0.578921,0.278816,5188,11.158843,5.374248
3,jvm_jvm-impoove_full,exp_scale_fixed,0.037350,0.013179,198,18.863498,6.656050
4,jvm_jvm-impoove_full,exp_scale_pausing,0.578921,0.269267,1165,49.692769,23.113071
5,jvm_jvm-impoove_full,exp_scale_rampup,0.822756,0.404496,399900,0.205740,0.101149
6,jvm_jvm-impoove_full,exp_scale_shaped,0.578921,0.267598,15565,3.719375,1.719232
7,monolith_feature_monolith_full,exp_scale_fixed,0.010539,0.004126,162,6.505308,2.546625
8,monolith_feature_monolith_full,exp_scale_pausing,0.163348,0.078036,773,21.131733,10.095230
9,monolith_feature_monolith_full,exp_scale_rampup,0.259978,0.107194,13922,1.867387,0.769963


In [121]:
def agg(x):
    return f"{x.min():.2f} - {x.max():.2f}"


main_table = pods_mean_cost_per_request[pods_mean_cost_per_request["exp_branch"].isin(dep_focus)].groupby("exp_branch")[["requested_cost","used_cost","used_cost_per_r"]].agg(agg).reset_index()
main_table["exp_branch"] = main_table["exp_branch"].apply(lambda x: lable_names[x])
main_table = main_table.rename(columns={"requested_cost":"Total Cost [$\$$]","used_cost":"Consumed Cost [$\$$]", "used_cost_per_r":"Cost per Request [$m\\textcent$]", "exp_branch":"Feature"})
with open("figs/cost_table_deployment.tex","w") as f:
    f.write(main_table.to_latex(index=False))
display(main_table)

main_table = pods_mean_cost_per_request[pods_mean_cost_per_request["exp_branch"].isin(ms_focus)].groupby("exp_branch")[["requested_cost","used_cost","used_cost_per_r"]].agg(agg).reset_index()
main_table["exp_branch"] = main_table["exp_branch"].apply(lambda x: lable_names[x])
main_table = main_table.rename(columns={"requested_cost":"Total Cost [$\$$]","used_cost":"Consumed Cost [$\$$]", "used_cost_per_r":"Cost per Request [$m\\textcent$]", "exp_branch":"Feature"})
with open("figs/cost_table_ms_improvment.tex","w") as f:
    f.write(main_table.to_latex(index=False))
display(main_table)

,Feature,Total Cost [$\$$],Consumed Cost [$\$$],Cost per Request [$m\textcent$]
0,Microservice Baseline,0.05 - 0.82,0.02 - 0.37,0.66 - 6.74
1,Monolith Variant,0.01 - 0.26,0.00 - 0.11,0.75 - 10.10
2,Serverless Variant,0.46 - 4.60,0.07 - 0.94,0.53 - 161.61


,Feature,Total Cost [$\$$],Consumed Cost [$\$$],Cost per Request [$m\textcent$]
0,Microservice Baseline,0.05 - 0.82,0.02 - 0.37,0.66 - 6.74
1,Service Reduction,0.06 - 0.87,0.02 - 0.43,0.28 - 24.98
2,Infrastructure Service Replacement,0.06 - 0.74,0.02 - 0.28,2.53 - 24.15
